In [ ]:
subscription_key = 'xxxxxxxxx'
assert subscription_key

In [ ]:
vision_endpoint = 'https://xxxxxxxxx.cognitiveservices.azure.com/'
vision_base_url = vision_endpoint + "vision/v1.0/"
vision_analyze_url = vision_base_url + "analyze"

In [ ]:
images_folder = "https://raw.githubusercontent.com/idiWork/Experiment_102/master/images/idiwork_experiment_102_sample_image_"
resort_pool = images_folder + "01.jpg"
resort_room = images_folder + "02.jpg"
resort_diner = images_folder + "03.jpg"
hotel_diner = images_folder + "04.jpg"
hotel_pool = images_folder + "05.jpg"
hotel_room = images_folder + "06.jpg"

In [ ]:
image_url = resort_room

In [ ]:
from IPython.display import Image, display
display(Image(image_url))

In [ ]:
import requests
headers  = {'Ocp-Apim-Subscription-Key': subscription_key }
params   = {'visualFeatures': 'Categories,Description,Tags,Color'}
data     = {'url': image_url}
response = requests.post(vision_analyze_url, headers=headers, params=params, json=data)
response.raise_for_status()
analysis = response.json()
analysis

In [ ]:
caption = analysis["description"]["captions"][0]["text"].capitalize()
caption

In [ ]:
topTags = analysis["description"]["tags"][0:4]
topTags

In [ ]:
text_analytics_subscription_key = 'xxxxxxxxx'
assert text_analytics_subscription_key

In [ ]:
text_analytics_base_url = 'https://xxxxxxxxx.cognitiveservices.azure.com/text/analytics/v2.0/'
sentiment_api_url = text_analytics_base_url + "sentiment"

In [ ]:
neg_sent = """
This is the worst stay I have every had! I have nothing positive to say about the diner and am extremely angry.
Poor service, bad food and too expensive drinks. I will not recommend it to anyone.
"""
pos_sent = """
I would like to thank you very much for a wonderful stay at the hotel. 
The room we stayed in was very nice and had plenty of room for the whole family and the beds were especially comfortable.
"""
neutral_sent = """
The kids go to the swimming pool all day while we take sunbaths near them.
We have an umbrellas and a deck chair for each of us if we go early in the morning. 
"""
long_review = """
We arrived in the evening and the reception was very busy, 
so we decided to wait until it was quieter. 
Unfortunately, we could not charge anything to the room yet as we were not checked in. 
We could not use the Wi-Fi either so had to use the guest Wi-Fi which was very slow and poor quality.
Once we checked in, the room was very nice and had fantastic views, but the bathroom was quite dirty.  
We called reception and they sent housekeeping who were very quick to clean the bathroom properly and even gave us some free bath toys for the kids which they loved. 
Overall the stay was pleasant and the service from staff was excellent. 
"""

In [ ]:
review_text = long_review

In [ ]:
documents = {'documents' : [
    {'id': '1', 'language': 'en', 'text': review_text}
]}

In [ ]:
headers   = {"Ocp-Apim-Subscription-Key": text_analytics_subscription_key}
response  = requests.post(sentiment_api_url, headers=headers, json=documents)
sentiments = response.json()
sentiments

In [ ]:
score = sentiments['documents'][0]['score']
score

In [ ]:
score_interpretation = "Neutral"
if (score < 0.4): 
    score_interpretation = "Negative"
elif (score >= 0.6):
    score_interpretation = "Positive"
score_interpretation

In [ ]:
def invoke_service(ml_service_key, ml_service_scoring_endpoint, ml_service_input):
    headers   = {"Authorization": "Bearer " + ml_service_key}
    response  = requests.post(ml_service_scoring_endpoint, headers=headers, json=ml_service_input)
    result = response.json()
    return result

In [ ]:
classifier_service_key = "" #leave this value empty if the service does not have authentication enabled
classifier_service_scoring_endpoint = 'http://xxx-xxx-xxx-xxx-xxx.westeurope.azurecontainer.io/score'
classifier_service_input = [review_text]

In [ ]:
classifier_result = invoke_service(classifier_service_key, classifier_service_scoring_endpoint, classifier_service_input)
classifier_result

In [ ]:
# Interpret the classifier result
classification = 'Room Review'
if classifier_result == 1:
    classification = 'Diner Review'
elif classifier_result == 2:
    classification = 'Pool Review'
classification

In [ ]:
summarizer_service_key = "" #leave this value empty if the service does not have authentication enabled
summarizer_service_scoring_endpoint = 'http://xxx-xxx-xxx-xxx-xxx.westeurope.azurecontainer.io/score'
summarizer_service_input = review_text

In [ ]:
summarizer_result = invoke_service(summarizer_service_key, summarizer_service_scoring_endpoint, summarizer_service_input)
formatted_result =  summarizer_result[0].replace("\\n", " ").strip() if len(summarizer_result) > 0 else "N/A"
formatted_result

In [ ]:
from IPython.core.display import HTML

displayTemplate = """
<div><h3>Claim Summary</h3></div>
<div>&nbsp;</div>
<div><b>Classification:</b> {}</div>
<div><b>Caption:</b> {}</div>
<div><b>Tags:</b> {}</div>
<div><b>Sentiment:</b> {}</div>
<div>&nbsp;</div>
<div><img src='{}' width='200px'></div>
<div>&nbsp;</div>
<div><b>Summary:</b></div>
<div>{}</div>
<div>&nbsp;</div>
<div><b>Claim:</b></div>
<div>{}</div>
"""
displayTemplate = displayTemplate.format(classification, caption, ' '.join(topTags), score_interpretation, image_url, formatted_result, review_text)
display(HTML(displayTemplate))